In [1]:
import sys
sys.path.append("../../src")
import os
import datetime
import pandas as pd
import numpy as np
from example_pendulum import get_pendulum_data
from sindy_utils import library_size
from training import train_network
import tensorflow as tf

# Generate data

In [2]:
training_data = get_pendulum_data(100)
validation_data = get_pendulum_data(10)

# Set up model and training parameters

In [3]:
params = {}

params['input_dim'] = training_data['x'].shape[-1]
params['latent_dim'] = 1
params['model_order'] = 2
params['poly_order'] = 3
params['include_sine'] = True
params['library_dim'] = library_size(2*params['latent_dim'], params['poly_order'], params['include_sine'], True)

# sequential thresholding parameters
params['sequential_thresholding'] = True
params['coefficient_threshold'] = 0.1
params['threshold_frequency'] = 500
params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))
params['coefficient_initialization'] = 'constant'

# loss function weighting
params['loss_weight_decoder'] = 1.0
params['loss_weight_sindy_x'] = 5e-4
params['loss_weight_sindy_z'] = 5e-5
params['loss_weight_sindy_regularization'] = 1e-5

params['activation'] = 'sigmoid'
params['widths'] = [128,64,32]

# training parameters
params['epoch_size'] = training_data['x'].shape[0]
params['batch_size'] = 1000
params['learning_rate'] = 1e-4

params['data_path'] = os.getcwd() + '/'
params['print_progress'] = True
params['print_frequency'] = 100

# training time cutoffs
params['max_epochs'] = 5000
params['refinement_epochs'] = 1001

# Run training experiments

In [4]:
num_experiments = 5
df = pd.DataFrame()
for i in range(num_experiments):
    print('EXPERIMENT %d' % i)

    params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))

    params['save_name'] = 'pendulum_' + datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S_%f")

    tf.compat.v1.reset_default_graph()

    results_dict = train_network(training_data, validation_data, params)
    df = pd.concat([df, pd.DataFrame([{**results_dict, **params}])], ignore_index=True)

df.to_pickle('experiment_results_' + datetime.datetime.now().strftime("%Y%m%d%H%M") + '.pkl')

EXPERIMENT 0
TRAINING


2024-06-03 22:40:42.649207: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled


Epoch 0
   training loss 0.013158665969967842, (0.008050801, 0.00022505404, 10.195771, 0.996659)
   validation loss 0.01769832707941532, (0.0083353445, 0.0006389415, 18.705967, 0.996659)
decoder loss ratio: 0.993749, decoder SINDy loss  ratio: 1.000000
Epoch 100
   training loss 0.01232650876045227, (0.007223421, 3.6008663e-05, 10.195771, 0.52005935)
   validation loss 0.016880638897418976, (0.007522451, 8.01487e-05, 18.705967, 0.52005935)
decoder loss ratio: 0.896835, decoder SINDy loss  ratio: 1.000000
Epoch 200
   training loss 0.012324335053563118, (0.0072259605, 0.00033027757, 10.195771, 0.047261674)
   validation loss 0.01686091348528862, (0.0075074094, 0.0009635908, 18.705967, 0.047261674)
decoder loss ratio: 0.895041, decoder SINDy loss  ratio: 1.000000
Epoch 300
   training loss 0.012324738316237926, (0.0072267745, 0.0015256598, 10.195771, 5.4546865e-05)
   validation loss 0.016857534646987915, (0.0075043375, 0.0042744474, 18.705967, 5.4546865e-05)
decoder loss ratio: 0.894675